In [19]:
import dill as pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras.models import load_model

In [28]:
def calculate_metrics_per_class(confusion_matrix):
    """
    Calculate sensitivity and specificity for each class from a confusion matrix.

    Parameters:
    - confusion_matrix: numpy array, shape (n_classes, n_classes)
        The confusion matrix.

    Returns:
    - sensitivities: numpy array, shape (n_classes,)
        Sensitivity (True Positive Rate or Recall) for each class.
    - specificities: numpy array, shape (n_classes,)
        Specificity (True Negative Rate) for each class.
    """
    n_classes = confusion_matrix.shape[0]
    sensitivities = np.zeros(n_classes)
    specificities = np.zeros(n_classes)

    for i in range(n_classes):
        true_positive = confusion_matrix[i, i]
        false_negative = np.sum(confusion_matrix[i, :]) - true_positive
        false_positive = np.sum(confusion_matrix[:, i]) - true_positive
        true_negative = np.sum(confusion_matrix) - (true_positive + false_negative + false_positive)

        # Sensitivity (True Positive Rate or Recall) for each class
        sensitivities[i] = true_positive / (true_positive + false_negative)

        # Specificity (True Negative Rate) for each class
        specificities[i] = true_negative / (true_negative + false_positive)

    return sensitivities, specificities

# Load the Cleaned test data

In [16]:
device_test_data = pd.read_csv("Cleaned_test.csv")
device_test_copy = device_test_data.copy()
device_test_copy.head()

,battery_power,blue,clock_speed,dual_sim,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,ram,talk_time,touch_screen,wifi,px_total,sc_diagonal,price_range
0,0.640978,1.007562,0.831049,-1.021836,0.959146,0.051047,-0.006963,1.03806,1.520803,1.335244,1.569322,0.365790,0.996490,0.989508,-0.909813,0.520941,3
1,0.827540,1.007562,1.566979,0.978631,0.959146,0.326778,0.686160,-0.74211,-0.661953,1.665305,-1.001709,1.647020,-1.003522,-1.010604,-0.649289,-0.653569,0
2,-1.672854,-0.992494,-0.395503,0.978631,0.959146,-1.327608,-0.353524,-0.82688,-0.661953,0.510093,0.423466,0.182757,-1.003522,-1.010604,-0.790720,-0.581328,1
3,-0.630834,1.007562,-0.150192,0.978631,0.959146,1.374555,0.339599,-0.74211,1.520803,-1.140209,0.606914,-0.732407,0.996490,0.989508,-0.588725,1.200917,2
4,0.488542,-0.992494,0.095118,0.978631,0.959146,1.098824,-0.700085,-1.25073,-1.098504,1.335244,0.229878,-0.183309,0.996490,0.989508,-0.796872,-0.766909,2


In [30]:
X = device_test_copy.drop(['price_range'], axis=1)
Y = device_test_copy['price_range']


# Load the ANN pre-trained mode

In [25]:
# To load the model back:
ANN_model = load_model('ANN_model')

#### Test the loaded model

In [29]:
# Predict Using ANN model
y_ANN_pred = np.argmax(ANN_model.predict(X), axis=1)
# Get the confusion matrix
conf_ANN_mat = cm(y_ANN_pred, Y)
print("Comfusion Matrix:")
print(conf_ANN_mat)

sensitivities, specificities = calculate_metrics_per_class(conf_ANN_mat)

# Print or use the metrics per class
for i in range(len(sensitivities)):
    print(f"Class {i + 1}: Sensitivity = {sensitivities[i]}, Specificity = {specificities[i]}")

# Calculate the accuracy: 
accuracy=np.trace(conf_ANN_mat)/np.sum(conf_ANN_mat)
print("The accuracy is: ", accuracy)


Comfusion Matrix:
[[ 88   4   0   0]
 [  3  81   7   0]
 [  0   6  82   6]
 [  0   0   8 113]]
Class 1: Sensitivity = 0.9565217391304348, Specificity = 0.9901960784313726
Class 2: Sensitivity = 0.8901098901098901, Specificity = 0.9674267100977199
Class 3: Sensitivity = 0.8723404255319149, Specificity = 0.9506578947368421
Class 4: Sensitivity = 0.9338842975206612, Specificity = 0.9783393501805054
The accuracy is:  0.914572864321608
